In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd

df_train = pd.read_csv("./data/train.csv", index_col=0)
df_test = pd.read_csv("./data/test.csv", index_col=0)
df_train.head(2)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [2]:
df_test.head(2)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, log_loss

# Feature pipeline
cat_features: list[str] = ["Pclass", "Sex", "SibSp", "Parch", "Embarked"]
trans = ColumnTransformer(
    [
        ("categories", OneHotEncoder(dtype="int", sparse=False, drop="first", handle_unknown="ignore"), cat_features),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)
# whole pipeline
pipe = Pipeline(
    [
        ("features", trans),
        ("model", RandomForestClassifier(random_state=42)),
    ]
)

# train
y_true = df_train["Survived"]
pipe.fit(X=df_train, y=y_true)
y_pred = pipe.predict(df_train)
print("precision score:", precision_score(y_true, y_pred))
print("recall score:", recall_score(y_true, y_pred))
print("log loss:", log_loss(y_true, y_pred))

# make submission
df_pred = pd.DataFrame(index=df_test.index.copy(), columns=['Survived'])
df_pred['Survived'] = pipe.predict(df_test)
df_pred.to_csv('./data/submission.csv', index=True)

/home/aleksei/.cache/pypoetry/virtualenvs/kaggle-20pbn8US-py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


precision score: 0.8716981132075472
recall score: 0.6754385964912281
log loss: 5.865689945479222


/home/aleksei/.cache/pypoetry/virtualenvs/kaggle-20pbn8US-py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Traceback (most recent call last):
  File "/home/aleksei/.cache/pypoetry/virtualenvs/kaggle-20pbn8US-py3.10/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/home/aleksei/.cache/pypoetry/virtualenvs/kaggle-20pbn8US-py3.10/lib/python3.10/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/home/aleksei/.cache/pypoetry/virtualenvs/kaggle-20pbn8US-py3.10/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /home/aleksei/.kaggle. Or use the environment method.


In [4]:
!poetry run kaggle competitions submit -c titanic -f ./data/submission.csv -m "Random forest"

100%|██████████████████████████████████████| 2.77k/2.77k [00:01<00:00, 1.98kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster